In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/My Drive/Colab Notebooks/db/spark/{}'
path

'/content/drive/My Drive/Colab Notebooks/db/spark/{}'

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

spark-3.1.1-bin-hadoop2.7/
spark-3.1.1-bin-hadoop2.7/NOTICE
spark-3.1.1-bin-hadoop2.7/kubernetes/
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/python_executable_check.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/autoscale.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/decommissioning.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/decommissioning_cleanup.py
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/spark/
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/spark/decom.sh
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/spark/entrypoint.sh
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/R/
spark-3.1.1-bin-hadoop2.7/kubernetes/docker

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [5]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import sys
from pyspark.sql.functions import count

In [6]:
# Install ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2021-06-07 01:00:40--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.200.34.95, 52.54.95.251, 54.236.206.131, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.200.34.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  16.5MB/s    in 0.8s    

2021-06-07 01:00:41 (16.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [7]:
# Configure Spark UI
conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
sc.stop()

In [8]:
# Create a URL through you can access the Spark UI
get_ipython().system_raw('./ngrok http 4050 &')

In [9]:
# Then wait for 10s to access the URL
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://c83143368073.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://c83143368073.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [10]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, Row
from pyspark.sql import SQLContext

In [14]:
spark = SparkContext(master = "local", appName="DataFrames")
sql_context = SQLContext(spark)

In [12]:
!head -n 10 {path.format('deportistaError.csv').replace(" ", "\ ")}

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,,,273
4,Edgar Lindenau Aabye,1,34,,,278
5,Christine Jacoba Aaftink,2,21,185,82,705
6,Per Knut Aaland,1,31,188,75,1096
7,John Aalberg,1,31,183,72,1096
8,"Cornelia ""Cor"" Aalten (-Strannood)",2,18,168,,705
9,Antti Sami Aalto,1,26,186,96,350


In [18]:
deportista_error_rdd = spark.textFile(path.format('deportistaError.csv')).map(lambda l : l.split(','))

In [16]:
def eliminaEncabezado(index, it):
  return iter(list(it)[1:])

In [19]:
deportista_error_rdd = deportista_error_rdd.mapPartitionsWithIndex(eliminaEncabezado)
deportista_error_rdd.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '', '', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '', '', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [20]:
deportista_error_rdd = deportista_error_df.map(lambda x : (x[0], x[1], x[2], x[3], x[4], x[5], x[6]))

schema = StructType([
    StructField('id', StringType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', StringType(), False),
    StructField('edad', StringType(), False),
    StructField('altura', StringType(), False),
    StructField('peso', StringType(), False),
    StructField('equipo_id', StringType(), False)
])

deportista_error_df = sql_context.createDataFrame(deportista_error_rdd, schema)

In [21]:
deportista_error_df.show()

+---+--------------------+------+----+------+----+---------+
| id|              nombre|genero|edad|altura|peso|equipo_id|
+---+--------------------+------+----+------+----+---------+
|  1|           A Dijiang|     1|  24|   180|  80|      199|
|  2|            A Lamusi|     1|  23|   170|  60|      199|
|  3| Gunnar Nielsen Aaby|     1|  24|      |    |      273|
|  4|Edgar Lindenau Aabye|     1|  34|      |    |      278|
|  5|Christine Jacoba ...|     2|  21|   185|  82|      705|
|  6|     Per Knut Aaland|     1|  31|   188|  75|     1096|
|  7|        John Aalberg|     1|  31|   183|  72|     1096|
|  8|"Cornelia ""Cor""...|     2|  18|   168|    |      705|
|  9|    Antti Sami Aalto|     1|  26|   186|  96|      350|
| 10|"Einar Ferdinand ...|     1|  26|      |    |      350|
| 11|  Jorma Ilmari Aalto|     1|  22|   182|76.5|      350|
| 12|   Jyri Tapani Aalto|     1|  31|   172|  70|      350|
| 13|  Minna Maarit Aalto|     2|  30|   159|55.5|      350|
| 14|Pirjo Hannele Aal..

In [27]:
from pyspark.sql.functions import udf

def conversionInt(val):
  return int(val) if len(val) > 0 else None

In [29]:
conversionInt_udf = udf(lambda z : conversionInt(z), IntegerType())
sql_context.udf.register('conversionInt_udf', conversionInt_udf)

<function __main__.<lambda>>

In [31]:
deportista_error_df.select(conversionInt_udf('altura'). alias('alturaUDF')).show()

+---------+
|alturaUDF|
+---------+
|      180|
|      170|
|     null|
|     null|
|      185|
|      188|
|      183|
|      168|
|      186|
|     null|
|      182|
|      172|
|      159|
|      171|
|     null|
|      184|
|      175|
|      189|
|     null|
|      176|
+---------+
only showing top 20 rows



In [32]:
from pyspark.storagelevel import StorageLevel

Guardar un DataFrame en cache 

In [33]:
deportista_error_df.is_cached

False

In [34]:
deportista_error_df.rdd.cache()

MapPartitionsRDD[35] at javaToPython at NativeMethodAccessorImpl.java:0

https://spark.apache.org/docs/2.4.6/api/python/pyspark.html#pyspark.StorageLevel

StorageLevel(useDisk, useMemory, useOffHeap, deserialized, replication)

In [35]:
deportista_error_df.rdd.getStorageLevel()

StorageLevel(False, True, False, False, 1)

Es necesario elminar la persistencia del rdd para indicar una nueva



In [38]:
deportista_error_df.rdd.unpersist()

MapPartitionsRDD[35] at javaToPython at NativeMethodAccessorImpl.java:0

In [39]:
deportista_error_df.rdd.persist(StorageLevel.MEMORY_AND_DISK_2)

MapPartitionsRDD[35] at javaToPython at NativeMethodAccessorImpl.java:0

Creación de un tipo de Nivel de almacenamiento propio

In [40]:
StorageLevel.MEMORY_AND_DISK_3 = StorageLevel(True, True, False, False, 3)

In [41]:
deportista_error_df.rdd.unpersist()

MapPartitionsRDD[35] at javaToPython at NativeMethodAccessorImpl.java:0

In [42]:
deportista_error_df.rdd.persist(StorageLevel.MEMORY_AND_DISK_3)

MapPartitionsRDD[35] at javaToPython at NativeMethodAccessorImpl.java:0

In [44]:
deportista_error_df.rdd.getStorageLevel()

StorageLevel(True, True, False, False, 3)

In [46]:
spark.stop()

##Particionado

In [47]:
spark = SparkSession.builder.appName('Particionado').master('local[5]').getOrCreate()

In [50]:
df = spark.range(0,20)
df.rdd.getNumPartitions()

5

In [51]:
rdd1 = spark.sparkContext.parallelize((0,20), 10)
rdd1.getNumPartitions()

10

In [52]:
rddFile = spark.sparkContext.textFile(path.format('deporte.csv'), 10).map(lambda l : l.split(''))

In [54]:
rddFile.getNumPartitions()

10

In [64]:
rddFile.saveAsTextFile('r')

Py4JJavaError: ignored